# Notebook 0
It is a notebook which shows how use DataSource, Data and EventFamily

In [1]:
import time
import pandas as pd
from IPython.core.display import display, HTML
from pprint import pprint
from datetime import datetime
from th2_data_services import Data
from th2_data_services.events_tree import EventsTree
from th2_data_services.provider.v5.data_source.http import HTTPProvider5DataSource
from th2_data_services.provider.v5.commands import http
from th2_data_services.filter import Filter
from th2_data_services.provider.v5.events_tree import EventsTreeCollectionProvider5, ParentEventsTreeCollectionProvider5
from pandas import DataFrame

# This settings for increase display jupyter notebook and dataframe table.
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.max_rows = 550
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', 1000)

/tmp/ipykernel_282136/3855546854.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### We set up interval for th2-rpt-data-provider

In [2]:
DEMO_HOST = "10.64.66.66" #"th2-kube-demo" # Host port where rpt-data-provider is located.
DEMO_PORT = "30999" # Node port of rpt-data-provider.

# Create DataSource class with host and route.
# You can change host and route using propeties.
data_source = HTTPProvider5DataSource(F"http://{DEMO_HOST}:{DEMO_PORT}")

# Note that DataSource exists for each version rpt-data-provider.

START_TIME = datetime(year=2021, month=6, day=20, hour=10, minute=44, second=41, microsecond=692724)
END_TIME = datetime(year=2021, month=6, day=20, hour=10, minute=45, second=49, microsecond=28579)

In [3]:
# For understand which event type on based name we get from stream.
def get_super_type(record):
    name = record.get("eventName")
    parent_id = record.get("parentEventId")
    super_type = record.get("eventType")
    if super_type:
        if "Recon" in name:
            super_type = "Recon Folder"
        else:
            if not parent_id:
                super_type = "Test Run"
            else:
                parent_event = collection.get_parent(parent_id)
                if parent_event:
                    parent_super_type = get_super_type(parent_event)
                    if parent_super_type == "Test Run":
                        super_type = "Test Case"
                    elif parent_super_type == "Recon Folder":
                        super_type = "Recon Rule"
                    elif parent_super_type == "Recon Rule":
                        super_type = "Recon Status"
                    elif parent_super_type == "Recon Status":
                        super_type = "Recon Event"

    return super_type

# Base extract (transform function)
# record is required arguments.
def extract_basic(record):
    new_object = {
            "super_type": get_super_type(record),
            "status": "SUCCESSFUL" if record.get("successful") else "FAILED",
            "body": record.get("body"),
            "parentEventId": record.get("parentEventId"),
            "eventId": record.get("eventId"),
            "eventName": record.get("eventName"),
            "body": record.get("body")
        }
    return new_object

# Checking ancestor (filter function)
# record is required arguments.
def is_test_case_ancestor(record):
    ancestor = collection.find_ancestor(record.get("eventId"), lambda record: "TS" in record.get("eventName"))
    if ancestor:
        return True
    return False

In [4]:
# You can change anything GetEvents. 
# Arguments gets based on route for th2-rpt-data-provider.
# This example shows pull the events.
events: Data = data_source.command(
    http.GetEvents(
        start_timestamp=START_TIME,
        end_timestamp=END_TIME,
        attached_messages=True,
        cache=True
    )
)

print("[INFO] The events have loaded.")

[INFO] The events have loaded.


### We build events tree for furtuher assistance

We build a collection for further assistance:
* collection = EventsTreeCollection(data) 

We passed data_source for the collection therefore unknown_events is recovered
* collection = EventsTreeCollection(data, data_source=data_source)

In [5]:
collection = EventsTreeCollectionProvider5(events, data_source=data_source)

print("[INFO] The collection has built!")

[INFO] The collection has built!


Using method `show()` we print view tree.

In [6]:
collection.show()

[TS_1]Aggressive IOC vs two orders: second order's price is lower than first
├── Case[TC_1.1]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR1
│   ├── Check sequence rule SessionKey{sessionAlias='demo-conn1', direction=FIRST} - STEP2: Trader "DEMO-CONN1" receives Execution Report. The order stands on book in status NEW
│   │   ├── Check messages
│   │   │   └── Verification 'ExecutionReport' message
│   │   ├── Check sequence (expected 1 / actual 1 , check order false)
│   │   └── Pre-filtering (filtered 1 / processed 1) messages
│   │       └── Verification 'ExecutionReport' message
│   ├── Check sequence rule SessionKey{sessionAlias='demo-conn1', direction=FIRST} - STEP4: Trader "DEMO-CONN1" receives Execution Report. The order stands on book in status NEW
│   │   ├── Check messages
│   │   │   └── Verification 'ExecutionReport' message
│   │   ├── Check sequence (expected 1 / actual 1 , check order false)
│   │   └── Pre-filtering (filtered 1 / processed 1) messages
│  

If you want to append a new event then use `append_event(event)` 

In [7]:
event = {
    "parentEventId": "6e3be13f-cab7-4653-8cb9-6e74fd95ade4:8c1114ac-d1b4-11eb-9278-591e568ad66e",
    "eventId": "test_id",
    "eventName": "Test Event"
}
collection.append_event(event)

You can also refer to the tree separately from the collection.

In [8]:
tree = collection.get_trees()[0]
tree.show()

[TS_1]Aggressive IOC vs two orders: second order's price is lower than first
├── Case[TC_1.1]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR1
│   ├── Check sequence rule SessionKey{sessionAlias='demo-conn1', direction=FIRST} - STEP2: Trader "DEMO-CONN1" receives Execution Report. The order stands on book in status NEW
│   │   ├── Check messages
│   │   │   └── Verification 'ExecutionReport' message
│   │   ├── Check sequence (expected 1 / actual 1 , check order false)
│   │   └── Pre-filtering (filtered 1 / processed 1) messages
│   │       └── Verification 'ExecutionReport' message
│   ├── Check sequence rule SessionKey{sessionAlias='demo-conn1', direction=FIRST} - STEP4: Trader "DEMO-CONN1" receives Execution Report. The order stands on book in status NEW
│   │   ├── Check messages
│   │   │   └── Verification 'ExecutionReport' message
│   │   ├── Check sequence (expected 1 / actual 1 , check order false)
│   │   └── Pre-filtering (filtered 1 / processed 1) messages
│  

You can get subtree by id use `get_subtree(event_id)` method in the collection or each tree..

In [9]:
tree.get_subtree("8d7ed4f1-d1b4-11eb-bae5-57b0c4472880").show()

placeOrderFIX demo-conn1 - STEP3: Trader "DEMO-CONN1" sends request to create passive Order with price lower than first order.
├── Checkpoint
│   ├── Checkpoint for session alias 'demo-conn1' direction 'FIRST' sequence '1624005455622011523'
│   ├── Checkpoint for session alias 'demo-conn1' direction 'SECOND' sequence '1624005455622140290'
│   ├── Checkpoint for session alias 'demo-conn2' direction 'FIRST' sequence '1624005448022245399'
│   ├── Checkpoint for session alias 'demo-conn2' direction 'SECOND' sequence '1624005448022426113'
│   ├── Checkpoint for session alias 'demo-dc1' direction 'FIRST' sequence '1624005475720919500'
│   ├── Checkpoint for session alias 'demo-dc1' direction 'SECOND' sequence '1624005475721015014'
│   ├── Checkpoint for session alias 'demo-dc2' direction 'FIRST' sequence '1624005466840263372'
│   ├── Checkpoint for session alias 'demo-dc2' direction 'SECOND' sequence '1624005466840347015'
│   ├── Checkpoint for session alias 'demo-log' direction 'FIRST' sequ

You can get leaves from all trees use `get_leaves()` method in the collection or each tree.

In [10]:
leaves = Data(tree.get_leaves())
for event in leaves.filter(lambda record: record.get("eventType") == "Verification"):
    pprint(event)

{'attachedMessageIds': [],
 'batchId': '53c1abd6-b2f9-4378-b061-434a3315c4a2',
 'endTimestamp': {'epochSecond': 1624185896, 'nano': 625264000},
 'eventId': '53c1abd6-b2f9-4378-b061-434a3315c4a2:8d722b53-d1b4-11eb-9278-591e568ad66e',
 'eventName': "Verification 'ExecutionReport' message",
 'eventType': 'Verification',
 'isBatched': True,
 'parentEventId': '8d722b52-d1b4-11eb-9278-591e568ad66e',
 'startTimestamp': {'epochSecond': 1624185896, 'nano': 205141000},
 'successful': True,
 'type': 'event'}
{'attachedMessageIds': [],
 'batchId': 'e71f316c-6912-43b1-a6cb-60c328a8239b',
 'endTimestamp': {'epochSecond': 1624185896, 'nano': 419542000},
 'eventId': 'e71f316c-6912-43b1-a6cb-60c328a8239b:8d71dd30-d1b4-11eb-9278-591e568ad66e',
 'eventName': "Verification 'ExecutionReport' message",
 'eventType': 'Verification',
 'isBatched': True,
 'parentEventId': '8d6f6c2f-d1b4-11eb-9278-591e568ad66e',
 'startTimestamp': {'epochSecond': 1624185896, 'nano': 214108000},
 'successful': True,
 'type': 'ev

{'attachedMessageIds': [],
 'batchId': '83775107-a1de-476c-a179-ef26b9313676',
 'endTimestamp': {'epochSecond': 1624185925, 'nano': 546761000},
 'eventId': '83775107-a1de-476c-a179-ef26b9313676:9eeaac2a-d1b4-11eb-9278-591e568ad66e',
 'eventName': "Verification 'ExecutionReport' message",
 'eventType': 'Verification',
 'isBatched': True,
 'parentEventId': '9e8078a0-d1b4-11eb-9278-591e568ad66e',
 'startTimestamp': {'epochSecond': 1624185924, 'nano': 847325000},
 'successful': True,
 'type': 'event'}
{'attachedMessageIds': [],
 'batchId': '0fcac3db-3747-421f-acd8-4e35921a74d3',
 'endTimestamp': {'epochSecond': 1624185931, 'nano': 28943000},
 'eventId': '0fcac3db-3747-421f-acd8-4e35921a74d3:a22f9e10-d1b4-11eb-9278-591e568ad66e',
 'eventName': "Verification 'ExecutionReport' message",
 'eventType': 'Verification',
 'isBatched': True,
 'parentEventId': 'a22f9e0f-d1b4-11eb-9278-591e568ad66e',
 'startTimestamp': {'epochSecond': 1624185931, 'nano': 19233000},
 'successful': True,
 'type': 'even

You can get full path to the event use `get_full_path(evet_id)` method in the colllection or each tree.

In [11]:
data = Data(tree.get_full_path("53c1abd6-b2f9-4378-b061-434a3315c4a2:8d722b53-d1b4-11eb-9278-591e568ad66e"))\
        .map(lambda record: {
            "eventName": record.get("eventName")
        }
    )
DataFrame(data)

,eventName
0,[TS_1]Aggressive IOC vs two orders: second order's price is lower than first
1,Case[TC_1.1]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for instrument INSTR1
2,"Check sequence rule SessionKey{sessionAlias='demo-conn1', direction=FIRST} - STEP2: Trader ""DEMO-CONN1"" receives Execution Report. The order stands on book in status NEW"
3,Check messages
4,Verification 'ExecutionReport' message


You can get the root id from all trees.

In [12]:
collection.get_roots_ids()

['84db48fc-d1b4-11eb-b0fb-199708acc7bc',
 '814422e1-9c68-11eb-8598-691ebd7f413d',
 '745f0318-d010-11eb-8e55-d3a76285d588',
 '5d0e9572-d047-11eb-bddd-1e8d42132387']

You can get the parent\children for event 

In [13]:
collection.get_parent("53c1abd6-b2f9-4378-b061-434a3315c4a2:8d722b53-d1b4-11eb-9278-591e568ad66e")

{'type': 'event',
 'eventId': '8d722b52-d1b4-11eb-9278-591e568ad66e',
 'batchId': None,
 'isBatched': False,
 'eventName': 'Check messages',
 'eventType': 'checkMessages',
 'endTimestamp': {'nano': 625252000, 'epochSecond': 1624185896},
 'startTimestamp': {'nano': 205141000, 'epochSecond': 1624185896},
 'parentEventId': '8d6e0c9e-d1b4-11eb-9278-591e568ad66e',
 'successful': True,
 'attachedMessageIds': []}

In [20]:
collection.get_children("8d722b52-d1b4-11eb-9278-591e568ad66e")

({'type': 'event',
  'eventId': '53c1abd6-b2f9-4378-b061-434a3315c4a2:8d722b53-d1b4-11eb-9278-591e568ad66e',
  'batchId': '53c1abd6-b2f9-4378-b061-434a3315c4a2',
  'isBatched': True,
  'eventName': "Verification 'ExecutionReport' message",
  'eventType': 'Verification',
  'endTimestamp': {'nano': 625264000, 'epochSecond': 1624185896},
  'startTimestamp': {'nano': 205141000, 'epochSecond': 1624185896},
  'parentEventId': '8d722b52-d1b4-11eb-9278-591e568ad66e',
  'successful': True,
  'attachedMessageIds': []},)

### Demo for events.

In [14]:
working_data: Data = data_source.command(
    http.GetEvents(
        start_timestamp=START_TIME,
        end_timestamp=END_TIME,
        attached_messages=True,
        cache=True
    )
)

for i in working_data:
    pprint(i)

{'attachedMessageIds': [],
 'batchId': None,
 'body': {},
 'endTimestamp': {'epochSecond': 1624185888, 'nano': 167676000},
 'eventId': '84db48fc-d1b4-11eb-b0fb-199708acc7bc',
 'eventName': "[TS_1]Aggressive IOC vs two orders: second order's price is "
              'lower than first',
 'eventType': '',
 'isBatched': False,
 'parentEventId': None,
 'startTimestamp': {'epochSecond': 1624185881, 'nano': 821639000},
 'successful': False,
 'type': 'event'}
{'attachedMessageIds': [],
 'batchId': None,
 'body': {},
 'endTimestamp': {'epochSecond': 1624185888, 'nano': 169710000},
 'eventId': '88a3ee80-d1b4-11eb-b0fb-199708acc7bc',
 'eventName': 'Case[TC_1.1]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for '
              'instrument INSTR1',
 'eventType': '',
 'isBatched': False,
 'parentEventId': '84db48fc-d1b4-11eb-b0fb-199708acc7bc',
 'startTimestamp': {'epochSecond': 1624185888, 'nano': 169672000},
 'successful': True,
 'type': 'event'}
{'attachedMessageIds': [],
 'batchId': None,
 'body': [{'

                                                                                  'key': False,
                                                                                  'operation': 'EQUAL',
                                                                                  'status': 'NA',
                                                                                  'type': 'collection'}},
                                                                 'key': False,
                                                                 'operation': 'EQUAL',
                                                                 'status': 'NA',
                                                                 'type': 'collection'}},
                                       'key': False,
                                       'operation': 'EQUAL',
                                       'status': 'NA',
                                       'type': 'collection'},
                      'Transa

                                           'status': 'PASSED',
                                           'type': 'field'},
                      'Side': {'actual': '1',
                               'expected': '1',
                               'key': False,
                               'operation': 'EQUAL',
                               'status': 'PASSED',
                               'type': 'field'},
                      'Text': {'actual': 'Simulated New Order Buy is placed',
                               'expected': '*',
                               'key': False,
                               'operation': 'NOT_EMPTY',
                               'status': 'PASSED',
                               'type': 'field'},
                      'TradingParty': {'actual': '1',
                                       'expected': '1',
                                       'fields': {'NoPartyIDs': {'actual': '4',
                                                                 '

                                                                                  'type': 'collection'},
                                                                            '3': {'actual': '3',
                                                                                  'expected': '3',
                                                                                  'fields': {'PartyID': {'actual': '0',
                                                                                                         'expected': '0',
                                                                                                         'key': False,
                                                                                                         'operation': 'EQUAL',
                                                                                                         'status': 'PASSED',
                                                                                      

                                          'key': False,
                                          'operation': 'EQUAL',
                                          'status': 'PASSED',
                                          'type': 'field'},
                     'Side': {'actual': '1',
                              'expected': '2',
                              'fields': {},
                              'key': False,
                              'operation': 'EQUAL',
                              'status': 'NA',
                              'type': 'field'},
                     'Text': {'actual': 'The simulated order has been fully '
                                        'traded',
                              'expected': 'The simulated order has been '
                                          'partially traded',
                              'fields': {},
                              'key': False,
                              'operation': 'EQUAL',
                            

...

                                                        'operation': 'NOT_EQUAL',
                                                        'status': 'PASSED',
                                                        'type': 'field'},
                                            'SenderCompID': {'actual': 'FGW',
                                                             'expected': 'null',
                                                             'key': False,
                                                             'operation': 'EQUAL',
                                                             'status': 'NA',
                                                             'type': 'field'},
                                            'SendingTime': {'actual': '2021-06-20T10:45:30.284',
                                                            'expected': 'null',
                                                            'key': False,
                                            

 'startTimestamp': {'epochSecond': 1624185932, 'nano': 128621000},
 'successful': True,
 'type': 'event'}
{'attachedMessageIds': [],
 'batchId': '4db96126-683f-4659-aa21-dd1c389b5d08',
 'body': [],
 'endTimestamp': {'epochSecond': 1624185932, 'nano': 128694000},
 'eventId': '4db96126-683f-4659-aa21-dd1c389b5d08:a2d776ec-d1b4-11eb-9278-591e568ad66e',
 'eventName': "Checkpoint for session alias 'demo-log' direction 'FIRST' "
              "sequence '1624029363623063053'",
 'eventType': 'Checkpoint for session',
 'isBatched': True,
 'parentEventId': '4db96126-683f-4659-aa21-dd1c389b5d08:a2d776e2-d1b4-11eb-9278-591e568ad66e',
 'startTimestamp': {'epochSecond': 1624185932, 'nano': 128621000},
 'successful': True,
 'type': 'event'}
{'attachedMessageIds': [],
 'batchId': None,
 'body': [{'rows': {'AccountType': {'columns': {'fieldValue': '1'},
                                    'type': 'row'},
                    'ClOrdID': {'columns': {'fieldValue': '3291690'},
                             

                                                                                  'operation': 'EQUAL',
                                                                                  'status': 'NA',
                                                                                  'type': 'collection'},
                                                                            '3': {'actual': '3',
                                                                                  'expected': 'null',
                                                                                  'fields': {'PartyID': {'actual': '0',
                                                                                                         'expected': 'null',
                                                                                                         'key': False,
                                                                                                         'operation': 'EQUAL',
   

{'attachedMessageIds': [],
 'batchId': 'e9440dd3-0575-46a3-92c1-02657a8d65da',
 'body': [{'fields': {'AccountType': {'actual': '1',
                                      'expected': 'null',
                                      'key': False,
                                      'operation': 'EQUAL',
                                      'status': 'NA',
                                      'type': 'field'},
                      'ClOrdID': {'actual': '3291690',
                                  'expected': 'null',
                                  'key': False,
                                  'operation': 'EQUAL',
                                  'status': 'NA',
                                  'type': 'field'},
                      'CumQty': {'actual': '10',
                                 'expected': 'null',
                                 'key': False,
                                 'operation': 'EQUAL',
                                 'status': 'NA',
                    

                                       'fields': {'NoPartyIDs': {'actual': '4',
                                                                 'expected': 'null',
                                                                 'fields': {'0': {'actual': '3',
                                                                                  'expected': 'null',
                                                                                  'fields': {'PartyID': {'actual': 'DEMO-CONN2',
                                                                                                         'expected': 'null',
                                                                                                         'key': False,
                                                                                                         'operation': 'EQUAL',
                                                                                                         'status': 'NA',
               

### Demo for filter, map and sift.

In [15]:
data: Data = working_data \
    .map(extract_basic) \
    .filter(lambda record: record.get("super_type") == "Verification")\
    .sift(limit=2)

for i in data:
    pprint(i)

{'body': [{'fields': {'AccountType': {'actual': '1',
                                      'expected': '1',
                                      'key': False,
                                      'operation': 'EQUAL',
                                      'status': 'PASSED',
                                      'type': 'field'},
                      'ClOrdID': {'actual': '9601585',
                                  'expected': '9601585',
                                  'key': True,
                                  'operation': 'EQUAL',
                                  'status': 'PASSED',
                                  'type': 'field'},
                      'CumQty': {'actual': '0',
                                 'expected': '0',
                                 'key': False,
                                 'operation': 'EQUAL',
                                 'status': 'PASSED',
                                 'type': 'field'},
                      'ExecID': {'actual'

In [16]:
data = working_data \
    .map(extract_basic) \
    .filter(lambda record: record.get("parentEventId"))\
    .filter(is_test_case_ancestor)\
    .sift(limit=2)

for i in data:
    pprint(i)

{'body': {},
 'eventId': '88a3ee80-d1b4-11eb-b0fb-199708acc7bc',
 'eventName': 'Case[TC_1.1]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for '
              'instrument INSTR1',
 'parentEventId': '84db48fc-d1b4-11eb-b0fb-199708acc7bc',
 'status': 'SUCCESSFUL',
 'super_type': ''}
{'body': [{'data': 'STEP1: Trader "DEMO-CONN1" sends request to create passive '
                   'Order.',
           'type': 'message'}],
 'eventId': '8bc787fe-d1b4-11eb-bae5-57b0c4472880',
 'eventName': 'placeOrderFIX demo-conn1 - STEP1: Trader "DEMO-CONN1" sends '
              'request to create passive Order.',
 'parentEventId': '88a3ee80-d1b4-11eb-b0fb-199708acc7bc',
 'status': 'SUCCESSFUL',
 'super_type': 'placeOrderFIX'}


In [17]:
def transform_output(record):
    body = record.get("body")
    if body:
        cumqty = body[0].get("fields", {}).get("CumQty")
    
    new_obj = {
        "CumQty": cumqty
    }    
    return new_obj

data = working_data \
    .map(extract_basic) \
    .filter(lambda record: record.get("super_type") == "Verification")\
    .map(transform_output)\
    .sift(limit=5, skip=5)

for i in data:
    pprint(i)

{'CumQty': {'actual': '30',
            'expected': '30',
            'key': True,
            'operation': 'EQUAL',
            'status': 'PASSED',
            'type': 'field'}}
{'CumQty': {'actual': '10',
            'expected': 'null',
            'key': False,
            'operation': 'EQUAL',
            'status': 'NA',
            'type': 'field'}}
{'CumQty': {'actual': '30',
            'expected': 'null',
            'key': False,
            'operation': 'EQUAL',
            'status': 'NA',
            'type': 'field'}}
{'CumQty': {'actual': '10',
            'expected': '10',
            'key': True,
            'operation': 'EQUAL',
            'status': 'PASSED',
            'type': 'field'}}
{'CumQty': {'actual': '40',
            'expected': '40',
            'key': True,
            'operation': 'EQUAL',
            'status': 'PASSED',
            'type': 'field'}}


### Demo for messages.

In [18]:
working_data = data_source.command(
    http.GetMessages(
        start_timestamp=datetime(year=2021, month=6, day=15, hour=9, minute=44, second=41, microsecond=692724),
        end_timestamp=datetime(year=2021, month=6, day=15, hour=12, minute=45, second=49, microsecond=28579),
        stream=["th2-hand-demo"]
    )
)

for i in working_data.sift(limit=5):
    pprint(i)

{'attachedEventIds': [],
 'body': {'fields': {'messages': {'listValue': {'values': [{'messageValue': {'fields': {'ActionName': {'simpleValue': 'winOpen'},
                                                                                        'appFile': {'simpleValue': 'MiniFIX.exe'},
                                                                                        'workDir': {'simpleValue': 'C:\\Program '
                                                                                                                   'Files '
                                                                                                                   '(x86)\\MiniFIX\\'}}}},
                                                           {'messageValue': {'fields': {'ActionName': {'simpleValue': 'winSearchElement'},
                                                                                        'id': {'simpleValue': 'mainWindowElId'},
                                                     

 'body': {'fields': {'th2-hand-1': {'messageValue': {'fields': {'ErrorText': {'simpleValue': ''}}}},
                     'th2-hand-2': {'messageValue': {'fields': {'TextOut': {'simpleValue': ''}}}},
                     'th2-hand-3': {'messageValue': {'fields': {'ScriptOutputCode': {'simpleValue': 'SUCCESS'}}}},
                     'th2-hand-4': {'messageValue': {'fields': {'RhSessionId': {'simpleValue': '/Sesa2c385c3-c0e9-41d6-b856-9ec5bde8e2b5'}}}}},
          'metadata': {'id': {'connectionId': {'sessionAlias': 'th2-hand-demo'},
                              'sequence': '1623751720263566004',
                              'subsequence': [1, 2, 3, 4]},
                       'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand',
                       'timestamp': '2021-06-15T10:11:13.870751Z'}},
 'bodyBase64': 'eyJTY3JpcHRPdXRwdXRDb2RlIjoiU1VDQ0VTUyIsIkVycm9yVGV4dCI6IiIsIlRleHRPdXQiOiIiLCJSaFNlc3Npb25JZCI6Ii9TZXNhMmMzODVjMy1jMGU5LTQxZDYtYjg1Ni05ZWM1YmRlOGUyYjUifQ==',
 'direction': 

In [19]:
def transform_output(record):
    new_obj = {
        "messageType": record.get("messageType")
    }
    return new_obj

data = working_data.map(transform_output)

for i in data:
    pprint(i)

{'messageType': 'th2-hand'}
{'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand'}
{'messageType': 'th2-hand'}
{'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand'}
{'messageType': 'th2-hand'}
{'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand'}
{'messageType': 'th2-hand'}
{'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand'}
{'messageType': 'th2-hand'}
{'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand'}
{'messageType': 'th2-hand'}
{'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand'}
{'messageType': 'th2-hand'}
{'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand'}
{'messageType': 'th2-hand'}
{'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand'}
{'messageType': 'th2-hand'}
{'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand'}
{'messageType': 'th2-hand'}
{'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand'}
{'messageType': 'th2-hand'}
{'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand'}
{'messageType': 'th2-hand'}
{'messageType': 'th2-hand/th2-hand/th2-hand/th2-hand'}
{'me